In [1]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import torchvision
from torchvision import datasets, models, transforms
import time
import os

import sampler

In [2]:
dataTrans = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
        ])
 
    # image data path
data_dir = './data/aifood/images'
all_image_datasets = datasets.ImageFolder(data_dir, dataTrans)
    
trainsize = int(0.8*len(all_image_datasets))
testsize = len(all_image_datasets) - trainsize
train_dataset, test_dataset = torch.utils.data.random_split(all_image_datasets,[trainsize,testsize])
    
image_datasets = {'train':train_dataset,'val':test_dataset}
    

    # wrap your data and label into Tensor
dataloders = {
    'train':torch.utils.data.DataLoader(train_dataset,
                                        batch_size=64,
                                        num_workers=4,
                                        sampler=sampler.ImbalancedDatasetSampler(train_dataset.dataset)
                                                      ),
     'val':torch.utils.data.DataLoader(test_dataset,
                                        batch_size=64,
                                              shuffle=True,
                                              num_workers=4)
            }
    
dataloders = {x: torch.utils.data.DataLoader(image_datasets[x],
                                                 batch_size=64,
                                                 shuffle=True,
                                                 num_workers=4) for x in ['train', 'val']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

    # use gpu or not
use_gpu = torch.cuda.is_available()

In [3]:
def train_model(model, lossfunc, optimizer, scheduler, num_epochs=10):
    start_time = time.time()

    best_model_wts = model.state_dict()
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train(True)  # Set model to training mode
            else:
                model.train(False)  # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0.0

            # Iterate over data.
            for data in dataloders[phase]:
                # get the inputs
                inputs, labels = data

                # wrap them in Variable
                if use_gpu:
                    inputs = Variable(inputs.cuda())
                    labels = Variable(labels.cuda())
                else:
                    inputs, labels = Variable(inputs), Variable(labels)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                outputs = model(inputs)
                _, preds = torch.max(outputs.data, 1)
                loss = lossfunc(outputs, labels)

                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                # statistics
                running_loss += loss.data
                running_corrects += torch.sum(preds == labels.data).to(torch.float32)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()

    elapsed_time = time.time() - start_time
    print('Training complete in {:.0f}m {:.0f}s'.format(
        elapsed_time // 60, elapsed_time % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
  
    return model


In [4]:
 # get model and replace the original fc layer with your fc layer
model_ft = models.resnet50(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 75)

if use_gpu:
    model_ft = model_ft.cuda()

    # define loss function
lossfunc = nn.CrossEntropyLoss()

    # setting optimizer and trainable parameters
 #   params = model_ft.parameters()
 # list(model_ft.fc.parameters())+list(model_ft.layer4.parameters())
params = list(model_ft.fc.parameters())+list( model_ft.parameters())
optimizer_ft = optim.SGD(params, lr=0.001, momentum=0.9)

    # Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

model_ft = train_model(model=model_ft,
                           lossfunc=lossfunc,
                           optimizer=optimizer_ft,
                           scheduler=exp_lr_scheduler,
                           num_epochs=20)

Epoch 0/19
----------
train Loss: 0.0259 Acc: 0.6350
val Loss: 0.0112 Acc: 0.8142
Epoch 1/19
----------
train Loss: 0.0088 Acc: 0.8549
val Loss: 0.0081 Acc: 0.8656
Epoch 2/19
----------
train Loss: 0.0055 Acc: 0.9109
val Loss: 0.0069 Acc: 0.8842
Epoch 3/19
----------
train Loss: 0.0036 Acc: 0.9466
val Loss: 0.0064 Acc: 0.8944
Epoch 4/19
----------
train Loss: 0.0023 Acc: 0.9717
val Loss: 0.0063 Acc: 0.8932
Epoch 5/19
----------
train Loss: 0.0015 Acc: 0.9834
val Loss: 0.0062 Acc: 0.8971
Epoch 6/19
----------
train Loss: 0.0010 Acc: 0.9910
val Loss: 0.0062 Acc: 0.8973
Epoch 7/19
----------
train Loss: 0.0008 Acc: 0.9942
val Loss: 0.0061 Acc: 0.9009
Epoch 8/19
----------
train Loss: 0.0007 Acc: 0.9945
val Loss: 0.0061 Acc: 0.9018
Epoch 9/19
----------
train Loss: 0.0007 Acc: 0.9945
val Loss: 0.0061 Acc: 0.9022
Epoch 10/19
----------
train Loss: 0.0007 Acc: 0.9945
val Loss: 0.0061 Acc: 0.9016
Epoch 11/19
----------
train Loss: 0.0007 Acc: 0.9946
val Loss: 0.0061 Acc: 0.9020
Epoch 12/19
--

In [8]:
torch.save(model_ft.state_dict(), './model75.pth') 
